# Import Modules

In [1]:
import os
import sys
import pickle

import numpy as np
import pandas as pd

# #############################################################################
sys.path.insert(0, os.path.join(os.environ["PROJ_irox"], "data"))
from proj_data_irox import (
    bulk_dft_data_path, unique_ids_path,
    static_irox_structures_path)

# #############################################################################
from IPython.display import display

In [2]:
from out_data.inputs_nersc_iro2 import ignore_ids as ignore_ids_nersc_iro2
# from out_data.inputs_nersc_iro3 import ignore_ids as ignore_ids_nersc_iro3
ignore_ids_nersc_iro3 = []

from out_data.inputs_sher_iro2 import ignore_ids as ignore_ids_sher_iro2
from out_data.inputs_sher_iro3 import ignore_ids as ignore_ids_sher_iro3

from out_data.inputs_slac_iro2 import ignore_ids as ignore_ids_slac_iro2
# from out_data.inputs_slac_iro3 import ignore_ids as ignore_ids_slac_iro3
ignore_ids_slac_iro3 = []

ignore_ids_iro2 = ignore_ids_nersc_iro2 + ignore_ids_sher_iro2 + ignore_ids_slac_iro2
ignore_ids_iro3 = ignore_ids_nersc_iro3 + ignore_ids_sher_iro3 + ignore_ids_slac_iro3

print("len(ignore_ids_iro2):", len(ignore_ids_iro2))
print("len(ignore_ids_iro3):", len(ignore_ids_iro3))

ignore_ids_dict = {
    "AB2": ignore_ids_iro2,
    "AB3": ignore_ids_iro3}


# Pickling data ######################################################
import os; import pickle
directory = "out_data"
if not os.path.exists(directory): os.makedirs(directory)
with open(os.path.join(directory, "ignore_ids.pickle"), "wb") as fle:
    pickle.dump(ignore_ids_dict, fle)
# #####################################################################

len(ignore_ids_iro2): 191
len(ignore_ids_iro3): 0


In [3]:
from out_data.ids_to_run_nersc_iro2 import ids_to_run as ids_to_run_nersc_iro2
from out_data.ids_to_run_nersc_iro3 import ids_to_run as ids_to_run_nersc_iro3

from out_data.ids_to_run_slac_iro2 import ids_to_run as ids_to_run_slac_iro2
# from out_data.ids_to_run_slac_iro3 import ids_to_run as ids_to_run_slac_iro3
ids_to_run_slac_iro3 = []

from out_data.ids_to_run_sher_iro2 import ids_to_run as ids_to_run_sher_iro2
from out_data.ids_to_run_sher_iro3 import ids_to_run as ids_to_run_sher_iro3

ids_to_run_iro2 = ids_to_run_nersc_iro2 + ids_to_run_sher_iro2 + ids_to_run_slac_iro2
ids_to_run_iro3 = ids_to_run_nersc_iro3 + ids_to_run_sher_iro3 + ids_to_run_slac_iro3

ids_to_run_iro3 = list(set(ids_to_run_iro3))
ids_to_run_iro2 = list(set(ids_to_run_iro2))

ids_to_run_sher = ids_to_run_sher_iro2 + ids_to_run_sher_iro3
ids_to_run_nersc = ids_to_run_nersc_iro2 + ids_to_run_nersc_iro3
ids_to_run_slac = ids_to_run_slac_iro2 + ids_to_run_slac_iro3

ids_to_run_sher = list(set(ids_to_run_sher))
ids_to_run_nersc = list(set(ids_to_run_nersc))
ids_to_run_slac = list(set(ids_to_run_slac))

# Read Data

In [4]:
with open(bulk_dft_data_path, "rb") as fle:
    df_bulk_dft = pickle.load(fle)
    # df_bulk_dft = df_bulk_dft[(df_bulk_dft["source"] == "raul")]

with open(static_irox_structures_path, "rb") as fle:
    df_static = pickle.load(fle)
    df_static = df_static[(df_static["source"] == "chris")]

df_ids = pd.read_csv(unique_ids_path)

with open("out_data/df_new_jobs.pickle", "rb") as fle:
    df_new_jobs = pickle.load(fle)

In [5]:
df_m = df_bulk_dft
df_m[df_m["id_old"] == 70]

# bulk_dft_data_path

,atoms,energy_pa,form_e_chris,id,id_old,path,source,stoich,energy,num_atoms
id_unique,,,,,,,,,,
7av2zgmqvp,"(Atom('O', [4.20334, 0.29899, 0.193], index=0)...",-6.201404,NaN,NaN,70,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,raul,AB3,-223.250561,36


In [6]:
def method(row_i):
    atoms = row_i["atoms"]
    num_atoms = atoms.get_number_of_atoms()
    return(num_atoms)

df_static["num_atoms"] = df_static.apply(method, axis=1)
df_static = df_static.drop(["atoms", "path", "source",], axis=1)

In [7]:
row_i = df_static.iloc[1]

id_old = row_i["id_old"]

id_old in ids_to_run_nersc

row_i
stoich_i = row_i["stoich"]
id_old = row_i["id_old"]


df_new_jobs_i = df_new_jobs[
    (df_new_jobs["stoich"] == stoich_i) & \
    (df_new_jobs["id"] == str(id_old).zfill(3))
    ]

assert len(df_new_jobs_i) == 1, "SIFISDF"

row_j = df_new_jobs_i.iloc[0]

action_j = row_j["action"]

bool_0 = action_j == 'Job is busy, will skip'
bool_1 = action_j == 'Time out or failed | Restarting isif 3 calc'
bool_2 = 'Time out or failed | Restarting isif 7 calc'
bool_3 = 'Job done | ISIF 3 done | --> isif 2'
if bool_0 or bool_1 or bool_2 or bool_3:
    tmp = 42

In [8]:
df_new_jobs["action"].unique().tolist()

[
 'ALL DONE! | ISIF 2',
 'Ignoring this id',
 'Job is busy, will skip',
 'Error, need manual attention',
 "Couldn't figure out what to do",
 'Time out or failed | Restarting isif 3 calc',
 'Time out or failed | Restarting isif 7 calc',
 'Job done | ISIF 3 done | --> isif 2',
]

[
 'ALL DONE! | ISIF 2',
 'Ignoring this id',
 'Job is busy, will skip',
 'Error, need manual attention',
 "Couldn't figure out what to do",
 'Time out or failed | Restarting isif 3 calc',
 'Time out or failed | Restarting isif 7 calc',
 'Job done | ISIF 3 done | --> isif 2',
]

"Error, need manual attention"
"Couldn't figure out what to do"


"Couldn't figure out what to do"

In [9]:
def method(row_i):
    new_column_values_dict = {}

    stoich_i = row_i["stoich"]
    id_old = row_i["id_old"]

    # #########################################################################
    if row_i.name in df_bulk_dft.index:
        new_column_values_dict["done"] = True
    else:
        new_column_values_dict["done"] = False

    # #########################################################################
    if stoich_i == "AB2":
        ignore_ids_i = ignore_ids_iro2
    elif stoich_i == "AB3":
        ignore_ids_i = ignore_ids_iro3
    # id_old = str(row_i["id_old"]).zfill(3)

    if str(row_i["id_old"]).zfill(3) in ignore_ids_i:
        new_column_values_dict["ignored"] = True
    else:
        new_column_values_dict["ignored"] = False

    # #########################################################################
    # stoich_i = row_i["stoich"] == "AB2"
    if stoich_i == "AB2":
        ids_to_run_i = ids_to_run_iro2
    elif stoich_i == "AB3":
        ids_to_run_i = ids_to_run_iro3

    if id_old in ids_to_run_i:
        new_column_values_dict["ids_to_run"] = True
    else:
        new_column_values_dict["ids_to_run"] = False


    # #########################################################################
    if stoich_i == "AB2":
        if id_old in ids_to_run_sher_iro2:
            new_column_values_dict["cluster"] = "sherlock"
        elif id_old in ids_to_run_nersc_iro2:
            new_column_values_dict["cluster"] = "nersc"
        elif id_old in ids_to_run_slac_iro2:
            new_column_values_dict["cluster"] = "slac"
        else:
            new_column_values_dict["cluster"] = np.nan

    elif stoich_i == "AB3":
        if id_old in ids_to_run_sher_iro3:
            new_column_values_dict["cluster"] = "sherlock"
        elif id_old in ids_to_run_nersc_iro3:
            new_column_values_dict["cluster"] = "nersc"
        elif id_old in ids_to_run_slac_iro3:
            new_column_values_dict["cluster"] = "slac"
        else:
            new_column_values_dict["cluster"] = np.nan


    # #########################################################################
    df_new_jobs_i = df_new_jobs[
        (df_new_jobs["stoich"] == stoich_i) & \
        (df_new_jobs["id"] == str(id_old).zfill(3))
        ]

    if id_old == 122:
        print(df_new_jobs_i)

    # assert len(df_new_jobs_i) == 1, (stoich_i, id_old)

    if len(df_new_jobs_i) == 1:
        row_j = df_new_jobs_i.iloc[0]

        action_j = row_j["action"]

        bool_0 = action_j == 'Job is busy, will skip'
        bool_1 = action_j == 'Time out or failed | Restarting isif 3 calc'
        bool_2 = action_j == 'Time out or failed | Restarting isif 7 calc'
        bool_3 = action_j == 'Job done | ISIF 3 done | --> isif 2'
        
        if id_old == 122:
            print(bool_0)
            print(bool_1)
            print(bool_2)
            print(bool_3)
            
        if bool_0 or bool_1 or bool_2 or bool_3:
            new_column_values_dict["running"] = True
        else:
            new_column_values_dict["running"] = False

        bool_0 = action_j == "Error, need manual attention"
        bool_1 = action_j == "Couldn't figure out what to do"
        if bool_0 or bool_1:
            new_column_values_dict["error"] = True

    elif len(df_new_jobs_i) == 0:
        new_column_values_dict["running"] = np.nan
    elif len(df_new_jobs_i) > 1:
        from IPython.display import display
        # print(df_new_jobs_i)
        display(df_new_jobs_i)
        new_column_values_dict["running"] = "TEMP, more than 1 sys"

    # #########################################################################
    for key, value in new_column_values_dict.items():
        row_i[key] = value
    return(row_i)

df_i = df_static
df_static = df_i.apply(method, axis=1)

                action   id  num_completed_isif_3  num_revisions  \
85  ALL DONE! | ISIF 2  122                     3              5   

                                             pre_path source stoich  
85  /global/cscratch1/sd/flores12/IrOx_Project_tem...  nersc    AB3  
False
False
False
False
                action   id  num_completed_isif_3  num_revisions  \
57  ALL DONE! | ISIF 2  122                     3              6   

                                             pre_path    source stoich  
57  /scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...  sherlock    AB2  
False
False
False
False


,action,id,num_completed_isif_3,num_revisions,pre_path,source,stoich
65,Ignoring this id,247,0,5,/global/cscratch1/sd/flores12/IrOx_Project_tem...,nersc,AB2
129,Ignoring this id,247,0,8,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,sherlock,AB2


,action,id,num_completed_isif_3,num_revisions,pre_path,source,stoich
64,Ignoring this id,246,0,5,/global/cscratch1/sd/flores12/IrOx_Project_tem...,nersc,AB2
128,Ignoring this id,246,0,10,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,sherlock,AB2


,action,id,num_completed_isif_3,num_revisions,pre_path,source,stoich
81,Ignoring this id,313,0,10,/global/cscratch1/sd/flores12/IrOx_Project_tem...,nersc,AB2
182,Ignoring this id,313,0,9,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,sherlock,AB2


,action,id,num_completed_isif_3,num_revisions,pre_path,source,stoich
72,ALL DONE! | ISIF 2,275,3,5,/global/cscratch1/sd/flores12/IrOx_Project_tem...,nersc,AB2
152,ALL DONE! | ISIF 2,275,3,6,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,sherlock,AB2


...

,action,id,num_completed_isif_3,num_revisions,pre_path,source,stoich
41,Ignoring this id,171,0,5,/global/cscratch1/sd/flores12/IrOx_Project_tem...,nersc,AB2
86,Ignoring this id,171,0,7,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,sherlock,AB2


,action,id,num_completed_isif_3,num_revisions,pre_path,source,stoich
49,ALL DONE! | ISIF 2,193,3,6,/global/cscratch1/sd/flores12/IrOx_Project_tem...,nersc,AB2
97,ALL DONE! | ISIF 2,193,3,6,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,sherlock,AB2


,action,id,num_completed_isif_3,num_revisions,pre_path,source,stoich
76,Ignoring this id,298,0,4,/global/cscratch1/sd/flores12/IrOx_Project_tem...,nersc,AB2
171,Ignoring this id,298,0,9,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,sherlock,AB2


,action,id,num_completed_isif_3,num_revisions,pre_path,source,stoich
68,Ignoring this id,256,0,4,/global/cscratch1/sd/flores12/IrOx_Project_tem...,nersc,AB2
135,Ignoring this id,256,0,21,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,sherlock,AB2


,action,id,num_completed_isif_3,num_revisions,pre_path,source,stoich
26,"Error, need manual attention",115,2,12,/global/cscratch1/sd/flores12/IrOx_Project_tem...,nersc,AB2
53,ALL DONE! | ISIF 2,115,3,6,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,sherlock,AB2


In [10]:
df_bulk_dft[df_bulk_dft["stoich"] == "AB2"].shape

(372, 10)

In [11]:
df_static.loc["9k7pvd7izl"]

[250, 168, 236, 280, 311, 22, 182]

# df_new_jobs_i =
df_new_jobs[
    (df_new_jobs["stoich"] == "AB2") & \
    (df_new_jobs["id"] == str(122).zfill(3))
    ]

,action,id,num_completed_isif_3,num_revisions,pre_path,source,stoich
57,ALL DONE! | ISIF 2,122,3,6,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,sherlock,AB2


In [12]:
df_static[
#     (df_static["done"] == True) & \
#     (df_static["ignored"] == True) & \
#     (df_static["num_atoms"] >= 100) & \
#     (df_static["ids_to_run"] == False) & \
#     (df_static["running"] == True) & \
    (df_static["error"] == True) & \
    (df_static["stoich"] == "AB3") & \
    [True for i in range(len(df_static))]
    ].shape

# ["id_old"].tolist()

(5, 10)

In [13]:
# total_ab2_systems = 
df_static[df_static["stoich"] == "AB2"].shape[0]

# Total AB2 systems
697

# #############################################################################
# Finished AB2 systems
283

# 100 atoms or greater
87

# Manually ignored
172

# Running jobs
56

56

In [14]:
# total_ab2_systems = 
df_static[df_static["stoich"] == "AB3"].shape[0]

# # Total AB2 systems
# 697

# # #############################################################################
# # Finished AB2 systems
# 283

# # 100 atoms or greater
# 87

# # Manually ignored
# 172

# # Running jobs
# 56

# # Errored
# 4

259

In [15]:
# Systems that have not been submitted yet
697 - (283 + 87 + 172 + 56 + 4)

95

In [16]:
# Total AB3

259

# Finished
246

# 100 atoms or greater
2

# Ignored
0

# Errored
5


# #############################################################################
246 + 2 + 5

253

In [17]:
all_ab2_ids = df_static[
    (df_static["stoich"] == "AB2") & \
    [True for i in range(len(df_static))]
    ].index.tolist()


ab2_completed_ids = df_static[
    (df_static["done"] == True) & \
#     (df_static["ignored"] == True) & \
#     (df_static["num_atoms"] >= 100) & \
#     (df_static["ids_to_run"] == False) & \
#     (df_static["running"] == True) & \
#     (df_static["error"] == True) & \
    (df_static["stoich"] == "AB2") & \
    [True for i in range(len(df_static))]
    ].index.tolist()

ab2_errored_ids = df_static[
#     (df_static["done"] == True) & \
#     (df_static["ignored"] == True) & \
#     (df_static["num_atoms"] >= 100) & \
#     (df_static["ids_to_run"] == False) & \
#     (df_static["running"] == True) & \
    (df_static["error"] == True) & \
    (df_static["stoich"] == "AB2") & \
    [True for i in range(len(df_static))]
    ].index.tolist()


ab2_large_atoms_ids = df_static[
#     (df_static["done"] == True) & \
#     (df_static["ignored"] == True) & \
    (df_static["num_atoms"] >= 100) & \
#     (df_static["ids_to_run"] == False) & \
#     (df_static["running"] == True) & \
#     (df_static["error"] == True) & \
    (df_static["stoich"] == "AB2") & \
    [True for i in range(len(df_static))]
    ].index.tolist()


ab2_ids_accounted_for = ab2_completed_ids + ab2_errored_ids + ab2_large_atoms_ids


ab2_ids_not_acc = [i for i in all_ab2_ids if i not in ab2_ids_accounted_for]
# [i for i all_ab3_ids]

# df_new_jobs

df_static.loc[ab2_ids_not_acc]

,cluster,done,error,id_old,ids_to_run,ignored,num_atoms,running,static_id,stoich
id_unique,,,,,,,,,,
8s9ecomu9g,NaN,False,NaN,443,False,False,18,NaN,fadaluvo_34,AB2
6pze7evdnf,NaN,False,NaN,329,False,False,24,NaN,safumeda_76,AB2
v5ckmsnqxi,nersc,False,NaN,247,True,True,66,"TEMP, more than 1 sys",palodutu_03,AB2
x49lcfvt9h,NaN,False,NaN,318,False,True,54,False,dubigudu_94,AB2
6qzs7tzuzh,NaN,False,NaN,341,False,False,12,NaN,wemukege_95,AB2
84m4v2bixy,NaN,False,NaN,578,False,False,24,NaN,henevane_85,AB2
zdxw9azjza,NaN,False,NaN,355,False,False,3,NaN,bohanafu_34,AB2
mwxdzp8g7u,NaN,False,NaN,642,False,False,12,NaN,getasuge_91,AB2
8snlxpnhmq,sherlock,False,NaN,544,True,False,45,True,betawako_67,AB2


In [18]:
all_ab3_ids = df_static[
    (df_static["stoich"] == "AB3") & \
    [True for i in range(len(df_static))]
    ].index.tolist()


ab3_completed_ids = df_static[
    (df_static["done"] == True) & \
#     (df_static["ignored"] == True) & \
#     (df_static["num_atoms"] >= 100) & \
#     (df_static["ids_to_run"] == False) & \
#     (df_static["running"] == True) & \
#     (df_static["error"] == True) & \
    (df_static["stoich"] == "AB3") & \
    [True for i in range(len(df_static))]
    ].index.tolist()

ab3_errored_ids = df_static[
#     (df_static["done"] == True) & \
#     (df_static["ignored"] == True) & \
#     (df_static["num_atoms"] >= 100) & \
#     (df_static["ids_to_run"] == False) & \
#     (df_static["running"] == True) & \
    (df_static["error"] == True) & \
    (df_static["stoich"] == "AB3") & \
    [True for i in range(len(df_static))]
    ].index.tolist()


ab3_large_atoms_ids = df_static[
#     (df_static["done"] == True) & \
#     (df_static["ignored"] == True) & \
    (df_static["num_atoms"] >= 100) & \
#     (df_static["ids_to_run"] == False) & \
#     (df_static["running"] == True) & \
#     (df_static["error"] == True) & \
    (df_static["stoich"] == "AB3") & \
    [True for i in range(len(df_static))]
    ].index.tolist()


ab3_ids_accounted_for = ab3_completed_ids + ab3_errored_ids + ab3_large_atoms_ids


ab3_ids_not_acc = [i for i in all_ab3_ids if i not in ab3_ids_accounted_for]
# [i for i all_ab3_ids]

# df_new_jobs

df_static.loc[ab3_ids_not_acc]

,cluster,done,error,id_old,ids_to_run,ignored,num_atoms,running,static_id,stoich
id_unique,,,,,,,,,,
95x1ctvenl,nersc,False,NaN,220,True,False,48,True,dahuroro_16,AB3
9amabibdx4,nersc,False,NaN,144,True,False,60,True,tinurahe_78,AB3
7u7k75mebp,nersc,False,NaN,46,True,False,64,True,lifowego_15,AB3


In [19]:
df_new_jobs[df_new_jobs["id"] == "070"]
# "7av2zgmqvp"

# df_bulk_dft[df_bulk_dft["id_old"] == 70]

df_m = df_bulk_dft
df_m[df_m["id_old"] == 70]

,atoms,energy_pa,form_e_chris,id,id_old,path,source,stoich,energy,num_atoms
id_unique,,,,,,,,,,
7av2zgmqvp,"(Atom('O', [4.20334, 0.29899, 0.193], index=0)...",-6.201404,NaN,NaN,70,/scratch/users/flores12/PROJ_irox_ml_oer/ml_bu...,raul,AB3,-223.250561,36
